In [2]:
import pandas as pd
import statsmodels.formula.api as smf
import numpy as np

data = pd.read_csv("/workspaces/bakery_sales_prediction/0_DataPreparation/00_data/data_with_lag/data_with_lag.csv")
#print(data.head())
#data brot:
df= data[data["Warengruppe_1"] == 1].copy()
data_wetter = pd.read_csv("/workspaces/bakery_sales_prediction/2_BaselineModel/02_Franz/weather_data_with_flags.csv")


df = df.drop(columns=["Warengruppe_1", "Warengruppe_2", "Warengruppe_3", "Warengruppe_4", "Warengruppe_5", "Warengruppe_6"])
df = df.drop(columns=["Datum"])
print(df)



          Umsatz  Temp_Very_Cold_current  Temp_Very_Cold_prev  \
0     148.828353                       0                  0.0   
1     159.793757                       0                  0.0   
2     111.885594                       0                  0.0   
3     168.864941                       0                  0.0   
4     171.280754                       0                  0.0   
...          ...                     ...                  ...   
1814  198.225523                       0                  0.0   
1815  214.470393                       0                  0.0   
1816  129.801991                       0                  0.0   
1817  148.541340                       0                  0.0   
1818  123.075682                       0                  0.0   

      Temp_Very_Cold_next  Temp_Cold_current  Temp_Cold_prev  Temp_Cold_next  \
0                     0.0                  0             0.0             0.0   
1                     0.0                  0             0.

In [3]:
# 1. Schritt 1: Untersuche lineare Korrelation zwischen den einzelnen Features mit der Zielvariable
# mit Korrelationsmatrix
import pandas as pd

# Zeilen- und Spaltenanzeige in Pandas erweitern
pd.set_option("display.max_rows", None)  # Zeigt alle Zeilen
pd.set_option("display.max_columns", None)  # Zeigt alle Spalten

# Korrelationsmatrix erstellen
correlation_matrix = df.corr()

# Korrelation mit der Zielvariable
correlation_with_target = correlation_matrix["Umsatz"].sort_values(ascending=False)
print(correlation_with_target)




Umsatz                            1.000000
Ostertag_current                  0.343797
Werktag_prev                      0.267577
is_holiday_current                0.261852
Samstag_current                   0.261361
Sommer_prev                       0.246935
Sonntag_next                      0.246710
Sommer_current                    0.244808
Sommer_next                       0.241408
is_holiday_prev                   0.233458
Freitag_prev                      0.231008
is_holiday_next                   0.230539
Werktag_current                   0.188045
Monat_8_next                      0.177330
Monat_8_current                   0.174328
Monat_8_prev                      0.165440
Ostertag_next                     0.162691
Monat_7_prev                      0.150243
Silvester_current                 0.144596
Mittwoch_prev                     0.144151
Monat_7_next                      0.138403
Monat_7_current                   0.136877
Feiertag_next                     0.136712
Donnerstag_

In [4]:
from sklearn.feature_selection import mutual_info_regression

# Nicht-lineare Zusammenhänge analysieren
X = df.drop("Umsatz", axis=1)
y = df["Umsatz"]
mi = mutual_info_regression(X, y)
mi_series = pd.Series(mi, index=X.columns).sort_values(ascending=False)
print(mi_series)


Sonntag_current                   0.176498
Samstag_prev                      0.173074
Montag_next                       0.168534
Werktag_current                   0.090059
VPI_next                          0.083656
VPI_current                       0.078067
VPI_prev                          0.074546
Werktag_prev                      0.068596
Sommer_prev                       0.049004
Temp_Cold_next                    0.048755
Winter_next                       0.047639
Sommer_next                       0.047247
Sommer_current                    0.043375
Temp_Cold_current                 0.042734
Winter_current                    0.042234
Winter_prev                       0.040589
Temp_Cold_prev                    0.039127
is_holiday_next                   0.038651
Werktag_next                      0.037550
is_holiday_current                0.035433
Heimspiel_prev                    0.032469
is_holiday_prev                   0.029717
Frühling_next                     0.029636
Frühling_cu

In [5]:

import numpy as np

# Korrelationsmatrix berechnen
corr_matrix = df.corr()

# Stark korrelierte Features (> 0.5) finden
high_corr = corr_matrix.abs() > 0.7
np.fill_diagonal(high_corr.values, False)  # Hauptdiagonale ignorieren
correlated_features = [column for column in high_corr.columns if any(high_corr[column])]
print(f"Stark korrelierte Features:", correlated_features)

Stark korrelierte Features: ['Temp_Cold_current', 'Temp_Cold_prev', 'Temp_Cold_next', 'Cloud_Cloudy_current', 'Cloud_Cloudy_prev', 'Cloud_Cloudy_next', 'Wind_Light_current', 'Wind_Light_prev', 'Wind_Light_next', 'Wind_Moderate_current', 'Wind_Moderate_prev', 'Wind_Moderate_next', 'Wind_Strong_current', 'Wind_Strong_prev', 'Wind_Strong_next', 'KielerWoche_current', 'KielerWoche_prev', 'KielerWoche_next', 'Montag_current', 'Montag_prev', 'Montag_next', 'Dienstag_current', 'Dienstag_prev', 'Dienstag_next', 'Mittwoch_current', 'Mittwoch_prev', 'Mittwoch_next', 'Donnerstag_current', 'Donnerstag_prev', 'Donnerstag_next', 'Freitag_current', 'Freitag_prev', 'Freitag_next', 'Samstag_current', 'Samstag_prev', 'Samstag_next', 'Sonntag_current', 'Sonntag_prev', 'Sonntag_next', 'VPI_current', 'VPI_prev', 'VPI_next', 'Number_of_ships_current', 'Number_of_ships_prev', 'Number_of_ships_next', 'Ship_current', 'Ship_prev', 'Ship_next', 'is_holiday_current', 'is_holiday_prev', 'is_holiday_next', 'Weihnac

In [6]:
from statsmodels.stats.outliers_influence import variance_inflation_factor

# VIF für alle Features berechnen
X = df.drop("Umsatz", axis=1)
vif = pd.DataFrame()
vif["Feature"] = X.columns
vif["VIF"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
print(vif)



/workspaces/bakery_sales_prediction/.venv/lib/python3.12/site-packages/statsmodels/stats/outliers_influence.py:197: RuntimeWarning: divide by zero encountered in scalar divide
  vif = 1. / (1. - r_squared_i)


                            Feature           VIF
0            Temp_Very_Cold_current  2.942845e+08
1               Temp_Very_Cold_prev           inf
2               Temp_Very_Cold_next           inf
3                 Temp_Cold_current           inf
4                    Temp_Cold_prev  2.404989e+07
5                    Temp_Cold_next           inf
6                 Temp_Mild_current           inf
7                    Temp_Mild_prev           inf
8                    Temp_Mild_next           inf
9                 Temp_Warm_current           inf
10                   Temp_Warm_prev           inf
11                   Temp_Warm_next           inf
12                 Temp_Hot_current           inf
13                    Temp_Hot_prev           inf
14                    Temp_Hot_next           inf
15              Cloud_Clear_current  5.022898e+05
16                 Cloud_Clear_prev           inf
17                 Cloud_Clear_next           inf
18      Cloud_Partly_Cloudy_current           inf


In [7]:
# suche miteinander korrelierrende features

In [8]:

# Korrelationsmatrix berechnen
corr_matrix = df.corr()

# Schwellenwert für "stark korreliert"
threshold = 0.4

# Finde die Paare mit starker Korrelation
strong_corr_pairs = set()

# Iteriere über die obere Dreiecksmatrix der Korrelationsmatrix
for i in range(len(corr_matrix.columns)):
    for j in range(i):
        if abs(corr_matrix.iloc[i, j]) > threshold:  # Korrelation über dem Schwellenwert
            strong_corr_pairs.add((corr_matrix.columns[i], corr_matrix.columns[j]))

# Anzeigen der Paare starker Korrelationen
print("Stark korrelierte Feature-Paare:")
print(strong_corr_pairs)


Stark korrelierte Feature-Paare:
{('VPI_next', 'VPI_current'), ('wetter_sehr_schön_prev', 'wetter_sehr_schön_current'), ('Sommer_next', 'Temp_Warm_prev'), ('Temp_Mild_next', 'Temp_Mild_prev'), ('Monat_6_next', 'KielerWoche_next'), ('is_holiday_prev', 'is_holiday_current'), ('Monat_10_next', 'Monat_10_prev'), ('Monat_6_next', 'KielerWoche_current'), ('Monat_5_prev', 'Frühling_next'), ('Monat_9_prev', 'Monat_9_current'), ('Monat_5_next', 'Monat_5_current'), ('Winter_next', 'Winter_current'), ('Monat_6_prev', 'KielerWoche_current'), ('is_holiday_next', 'is_holiday_prev'), ('Monat_12_prev', 'Winter_current'), ('Werktag_next', 'Samstag_next'), ('Monat_11_prev', 'Herbst_next'), ('Werktag_prev', 'Dienstag_next'), ('Monat_11_prev', 'Monat_11_current'), ('Monat_7_current', 'Sommer_current'), ('Temp_Hot_prev', 'Temp_Hot_current'), ('Monat_1_prev', 'Winter_prev'), ('zwischen_den_jahren_prev', 'Silvester_next'), ('Monat_10_next', 'Herbst_prev'), ('Monat_6_current', 'Sommer_prev'), ('Heimspiel_next